In [1]:
import numpy as np
import pandas as pd
import sys

# Matching
try:
    from scipy.sparse.csgraph import linear_sum_assignment
except (Exception) as e:
    print(e)
from scipy.spatial.distance import cdist

cannot import name 'linear_sum_assignment' from 'scipy.sparse.csgraph' (/home/mmaddox/.local/lib/python3.8/site-packages/scipy/sparse/csgraph/__init__.py)


# Matched Pairs Problem

Stable marriage problem

Minimum Weight Euclidean Matching (MWEM) 
Pefect matching between vectors in one group with another

## Weighted bipartite

Alternative to itertool's permutation function to create $\frac{n!}{k!(n-k)!}$ matches where k = 2. These pairings eliminate duplicate pairings (i.e. considers 1,2 the same as 2,1). This method was inefficient. DON'T USE.

In [2]:
def cartesian_product( x, y ):
    # Faster on smaller datasets (100, 70) but slower on larger datasets (500, 700)
    return np.hstack([ np.repeat(x, y.shape[0], axis=0),
               np.tile(y, (x.shape[0],1))] ).reshape(-1, 2, x.shape[1])

Developed a custom method as the libraries in existance are limited to provide only one matching solution even if there are multiple solutions of equal likelihood. However, this method is not scalable (algorithmically and memory inefficient) and crashes my computer on medium-sized datasets (e.g. two matrices of shape (500, 10))

In [3]:
from itertools import product, permutations

def match_pairs_custom(matrix_a, matrix_b):
    """
    return: list of row indices
    type: tuple(np.array, range)
    """
    #pairs = np.array(list(product(random_m, random_f))) # as opposed to custom cartesian_product
    dist = cdist(matrix_a, matrix_b, 'euclidean')
    if dist.shape[0] != dist.shape[1]:
        return None
    n = dist.shape[0]
    combinations = np.array(list(permutations(range(n))))
    summation = np.choose(combinations, dist).sum(axis = 1) # Increment through columns and grab rows which corerspond to array
    return combinations[np.where(summation == summation.min())] # respective column indices are all range(n)

If using **euclidean distance** ($\sqrt{(a_1-b_1)^2+(a_2-b_2)^2\dots+(a_n-b_n)^2}$), increasing distance values indicates decreasing similarity. 

If using **cosine distance** ($\frac{a^Tb}{|a||b|}$), increasing distance values indicaties increasing similarity. 

A quick side-by-side speed comparison (see below) showed that using cosine distance with maximization had the best scalability.

Scipy [linear_sum_assignment](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html) function is the best minimization algorithm for dense networks. Scipy [maximum_bipartite_matching](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.maximum_bipartite_matching.html) function uses [Hopcroft–Karp algorithm](https://epubs.siam.org/doi/10.1137/0202019) for maximization, however it does NOT allow for weights along edges.

In [4]:
def match_pairs( matrix_a, matrix_b ):
    """
    Determine the most optimal unique pairings between observations in matrix a with matrix b.
    
    @param: matrix_a:
    @type: np.array
    @param: matrix_b
    @type: np.array
    @return: (row indices simply increment )
    @type: tuple(np.array, np.array)
    """
    dist = cdist(matrix_a, matrix_b, 'euclidean')
    return linear_sum_assignment(dist)

[Hopcroft-Karp Algorithm](https://en.m.wikipedia.org/wiki/Hopcroft–Karp_algorithm)

$min(\sum_{i,j}d_{ij}x_{ij})\Longrightarrow O(E{\sqrt V})\approx O(V^\frac{3}{2})$ 

where $d_{ij}=distance~btw~V_i~and~V_j,$

$x_{ij}=\left\{\begin{matrix}1, & edge~belongs~to~matching \\ 0, & otherwise \end{matrix}\right.$

**Constraints**

1) Person $j$ paired with only one person from the other group: $\sum_jx_{ij}=1~for~1\le i\le n$

2) Person $i$ paired with only one person from the other group: $\sum_ix_{ij}=1~for~1\le j\le n$

3) Pairing between person $i$ and $j$ also means a pairing between $j$ and $i$: $x_{ij}\ge0 for 1\le i,j\le n$

In [6]:
from pulp import LpVariable, LpProblem, LpMinimize, value, lpSum, LpBinary, LpStatus

def match_pairs_custom(matrix_a, matrix_b):
    dist = cdist(matrix_a, matrix_b, 'euclidean') # (matrix_a.shape[0], matrix_b.shape[0])
    prob = LpProblem("Matching", LpMinimize)
    
    variable_names = [str(i)+str(j) for i in range(matrix_a.shape[0]) for j in range(matrix_b.shape[0])]
    variable_names.sort()
    
    relations = LpVariable.matrix("X", variable_names, cat = "Binary")
    allocation = np.array(relations).reshape(matrix_a.shape[0],matrix_b.shape[0])
    
    prob += lpSum(allocation*dist)
    
    for i in range(matrix_a.shape[0]):
        prob += lpSum([allocation[i,j] for j in range(matrix_b.shape[0])]) == 1
        prob += lpSum([allocation[j,i] for j in range(matrix_b.shape[0])]) == 1
        prob += lpSum(allocation[i,j] + allocation[j,i] for j in range(matrix_b.shape[0])) >= 0
        
    prob.solve()
    
    print( 'Status:\t', LpStatus[prob.status] )
    print( 'Objective Value:\t', prob.objective.value() )
    
    prob.writeLP("matching_prob.lp")
    
    return [(v.name, v.value()) for v in prob.variables()]

[A Survey on Algorithms for Euclidean Matching](https://courses.cs.duke.edu/fall08/cps234/projects/sayan_proj.pdf)

$O(n^\frac{3}{2}log(n))$

$max(\sum_i\alpha_i+\sum_j\beta_j)$ subject to $\alpha_i+\beta_j\le d_{ij}$ for $1\le i,j\le n$

https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0
    
https://scaron.info/blog/linear-programming-in-python-with-pulp.html
        
https://towardsdatascience.com/how-to-match-two-people-with-python-7583b51ff3f9
            
https://medium.com/opex-analytics/optimization-modeling-in-python-pulp-gurobi-and-cplex-83a62129807a

In [7]:
from pulp import LpVariable, LpProblem, LpMaximize, value, lpSum, LpStatus

def match_pairs(matrix_a, matrix_b):
    dist = cdist(matrix_a, matrix_b, 'euclidean') # (matrix_a.shape[0], matrix_b.shape[0])
    prob = LpProblem("Matching", LpMaximize)
    
    alpha = LpVariable.matrix( "alpha", [str(i) for i in range(matrix_a.shape[0])] )
    beta = LpVariable.matrix( "beta", [str(j) for j in range(matrix_b.shape[0])] )
    
    for i in range(matrix_a.shape[0]):
        for j in range(matrix_b.shape[0]):
            prob += alpha[i] + beta[j] <= dist[i,j]
            
    prob += lpSum(alpha) + lpSum(beta)
    prob.solve()
    
    print( 'Status:\t', LpStatus[prob.status] )
    print( 'Objective Value:\t', prob.objective.value() )
    
    prob.writeLP("matching_prob.lp")

    return np.isclose(np.array([a.value()+b.value() for a, b in product(alpha, beta)]).reshape(dist.shape), dist)

In [9]:
random_m = np.random.rand(100,2)
random_f = np.random.rand(100,2)

In [10]:
match_pairs(random_m, random_f)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mmaddox/.local/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/2261c87e8889499796e01cbcb5e82082-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/2261c87e8889499796e01cbcb5e82082-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10005 COLUMNS
At line 30206 RHS
At line 40207 BOUNDS
At line 40408 ENDATA
Problem MODEL has 10000 rows, 200 columns and 20000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 10000 (0) rows, 200 (0) columns and 20000 (0) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 0 ( 0%) - largest zero change 0
0  Obj -0 Dual inf 1.99998 (200) w.o. free dual inf (0)
0  Obj -0 Dual inf 1.99998 (200) w.o. free dual inf (0)
275  Obj 10.351206 Dual inf 16.999999 (11)
316  Obj 10.574242
Optimal - objective value 10.574242
Optimal objec

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Neither of these package functions were loading correctly. Therefore did math by hand.

## Non bipartite